In [1]:
import pandas as pd 
import numpy as np
import re
import collections
import matplotlib.pyplot as plt

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
data = []
with open("train.ft.txt","r") as f:
    for i in f:
        data.append(i)

In [3]:
NB_WORDS = 50000  # Parameter indicating the number of words we'll put in the dictionary
NB_START_EPOCHS = 26  # Number of epochs we usually start to train with
BATCH_SIZE = 512  #

In [4]:
def get_labels(data) :
    labels = []
    for i in data :
        if(((i.split()[0]).replace("__label__",""))=='1'):
            labels.append([1,0])
        else :
            labels.append([0,1])
    return labels

In [5]:
labels = get_labels(data)

In [6]:
(data[0].replace("__label__",""))[2:]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n'

In [7]:
def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 

In [ ]:
'!'.isalpha()

In [8]:
def remove_label(data):
    d = []
    for i in data :
        d.append((i.replace("__label__",""))[2:])
    return d

In [9]:
data = remove_label(data)

In [ ]:
data[10]

In [10]:
data = data[:20000]
labels = labels[:20000]

In [11]:
def cleaning_sentences(data):    
    for i in range(len(data)):
        data[i] = remove_stopwords(data[i])

In [12]:
cleaning_sentences(data)

In [ ]:
data[0]

In [13]:
data = np.array(data)
labels = np.array(labels)

In [14]:
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ", 
               char_level=False)
tk.fit_on_texts(data)

In [15]:
tokenizer = TreebankWordTokenizer()
training = []
for i in range(len(data)):
    training.append(tokenizer.tokenize(data[i]))
for i in range(len(training)):
    training[i] = [x.lower() for x in training[i]]

In [16]:
tk.word_index['<PAD>']=NB_WORDS+1
tk.word_index['<UNK>']=0
tk.word_index['n\'t'] = NB_WORDS+2

In [17]:
punct = ['!','"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', 
         '[', '/', ']', '^', '_', '`', '{', '|', '}', '~', '\t', '\n']
train_X =[]
for i in range(len(training)):
    sentence = []
    for j in range(len(training[i])):
        if training[i][j] in punct:
            pass
        else:
            sentence.append(training[i][j])
    train_X.append(sentence)

In [18]:
MAX_SEQ = 120
for s in range(len(train_X)):
    n = MAX_SEQ - len(train_X[s])
    if n < 0:
        train_X[s] = train_X[s][:MAX_SEQ]
    else:
        for i in range(n):
            train_X[s].append('<PAD>')
    for v in range(len(train_X[s])):
        if train_X[s][v] not in tk.word_index:
            train_X[s][v] = tk.word_index['<UNK>']
        else:
            train_X[s][v] = tk.word_index[train_X[s][v]]

In [19]:
reverse_word_map = dict(map(reversed, tk.word_index.items()))

In [20]:
train_X = np.array(train_X)
train_X.shape

(20000, 120)

In [25]:
embedding_size=32
model=Sequential()
model.add(Embedding(NB_WORDS+3, embedding_size, input_length=MAX_SEQ))
model.add(Bidirectional(LSTM(100, dropout=0.8)))
# model.add(Dense(20, kernel_regularizer=regularizers.l1_l2(0.01)))
model.add(Dense(2, activation='softmax'))
print(model.summary())

model.compile(optimizer='adam'
              , loss='binary_crossentropy'
              , metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 32)           1600096   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               106400    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 1,706,898
Trainable params: 1,706,898
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm

In [32]:
from __future__ import absolute_import
import numpy as np

from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.layers import Layer, InputSpec

from keras.utils.conv_utils import conv_output_length

import theano
import theano.tensor as T


def _dropout(x, level, noise_shape=None, seed=None):
    x = K.dropout(x, level, noise_shape, seed)
    x *= (1. - level) # compensate for the scaling by the dropout
    return x


class QRNN(Layer):
    '''Quasi RNN
    # Arguments
        units: dimension of the internal projections and the final output.
    # References
        - [Quasi-recurrent Neural Networks](http://arxiv.org/abs/1611.01576)
    '''
    def __init__(self, units, window_size=2, stride=1,
                 return_sequences=False, go_backwards=False, 
                 stateful=False, unroll=False, activation='tanh',
                 kernel_initializer='uniform', bias_initializer='zero',
                 kernel_regularizer=None, bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None, bias_constraint=None, 
                 dropout=0, use_bias=True, input_dim=None, input_length=None,
                 **kwargs):
        self.return_sequences = return_sequences
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.units = units 
        self.window_size = window_size
        self.strides = (stride, 1)

        self.use_bias = use_bias
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = dropout
        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.input_dim = input_dim
        self.input_length = input_length
        if self.input_dim:
            kwargs['input_shape'] = (self.input_length, self.input_dim)
        super(QRNN, self).__init__(**kwargs)

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        self.input_dim = input_shape[2]
        self.input_spec = InputSpec(shape=(batch_size, None, self.input_dim))
        self.state_spec = InputSpec(shape=(batch_size, self.units))

        self.states = [None]
        if self.stateful:
            self.reset_states()

        kernel_shape = (self.window_size, 1, self.input_dim, self.units * 3)
        self.kernel = self.add_weight(name='kernel',
                                      shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(name='bias', 
                                        shape=(self.units * 3,),
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)

        self.built = True

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        length = input_shape[1]
        if length:
            length = conv_output_length(length + self.window_size - 1,
                                        self.window_size, 'valid',
                                        self.strides[0])
        if self.return_sequences:
            return (input_shape[0], length, self.units)
        else:
            return (input_shape[0], self.units)

    def compute_mask(self, inputs, mask):
        if self.return_sequences:
            return mask
        else:
            return None

    def get_initial_states(self, inputs):
        # build an all-zero tensor of shape (samples, units)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        initial_state = K.tile(initial_state, [1, self.units])  # (samples, units)
        initial_states = [initial_state for _ in range(len(self.states))]
        return initial_states

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        if not self.input_spec:
            raise RuntimeError('Layer has never been called '
                               'and thus has no states.')

        batch_size = self.input_spec.shape[0]
        if not batch_size:
            raise ValueError('If a QRNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')

        if self.states[0] is None:
            self.states = [K.zeros((batch_size, self.units))
                           for _ in self.states]
        elif states is None:
            for state in self.states:
                K.set_value(state, np.zeros((batch_size, self.units)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                 'but it received ' + str(len(states)) +
                                 'state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if value.shape != (batch_size, self.units):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, self.units)) +
                                     ', found shape=' + str(value.shape))
                K.set_value(state, value)

    def __call__(self, inputs, initial_state=None, **kwargs):
        # If `initial_state` is specified,
        # and if it a Keras tensor,
        # then add it to the inputs and temporarily
        # modify the input spec to include the state.
        if initial_state is not None:
            if hasattr(initial_state, '_keras_history'):
                # Compute the full input spec, including state
                input_spec = self.input_spec
                state_spec = self.state_spec
                if not isinstance(state_spec, list):
                    state_spec = [state_spec]
                self.input_spec = [input_spec] + state_spec

                # Compute the full inputs, including state
                if not isinstance(initial_state, (list, tuple)):
                    initial_state = [initial_state]
                inputs = [inputs] + list(initial_state)

                # Perform the call
                output = super(QRNN, self).__call__(inputs, **kwargs)

                # Restore original input spec
                self.input_spec = input_spec
                return output
            else:
                kwargs['initial_state'] = initial_state
        return super(QRNN, self).__call__(inputs, **kwargs)

    def call(self, inputs, mask=None, initial_state=None, training=None):
        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            initial_states = inputs[1:]
            inputs = inputs[0]
        elif initial_state is not None:
            pass
        elif self.stateful:
            initial_states = self.states
        else:
            initial_states = self.get_initial_states(inputs)

        if len(initial_states) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_states)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        if self.unroll and input_shape[1] is None:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')
        constants = self.get_constants(inputs, training=None)
        preprocessed_input = self.preprocess_input(inputs, training=None)

        last_output, outputs, states = K.rnn(self.step, preprocessed_input,
                                            initial_states,
                                            go_backwards=self.go_backwards,
                                            mask=mask,
                                            constants=constants,
                                            unroll=self.unroll,
                                            input_length=input_shape[1])
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        # Properly set learning phase
        if 0 < self.dropout < 1:
            last_output._uses_learning_phase = True
            outputs._uses_learning_phase = True

        if self.return_sequences:
            return outputs
        else:
            return last_output

    def preprocess_input(self, inputs, training=None):
        if self.window_size > 1:
            inputs = K.temporal_padding(inputs, (self.window_size-1, 0))
        inputs = K.expand_dims(inputs, 2)  # add a dummy dimension

        output = K.conv2d(inputs, self.kernel, strides=self.strides,
                          padding='valid',
                          data_format='channels_last')
        output = K.squeeze(output, 2)  # remove the dummy dimension
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')

        if self.dropout is not None and 0. < self.dropout < 1.:
            z = output[:, :, :self.units]
            f = output[:, :, self.units:2 * self.units]
            o = output[:, :, 2 * self.units:]
            f = K.in_train_phase(1 - _dropout(1 - f, self.dropout), f, training=training)
            return K.concatenate([z, f, o], -1)
        else:
            return output

    def step(self, inputs, states):
        prev_output = states[0]

        z = inputs[:, :self.units]
        f = inputs[:, self.units:2 * self.units]
        o = inputs[:, 2 * self.units:]

        z = self.activation(z)
        f = f if self.dropout is not None and 0. < self.dropout < 1. else K.sigmoid(f)
        o = K.sigmoid(o)

        output = f * prev_output + (1 - f) * z
        output = o * output

        return output, [output]

    def get_constants(self, inputs, training=None):
        return []
 
    def get_config(self):
        config = {'units': self.units,
                  'window_size': self.window_size,
                  'stride': self.strides[0],
                  'return_sequences': self.return_sequences,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll,
                  'use_bias': self.use_bias,
                  'dropout': self.dropout,
                  'activation': activations.serialize(self.activation),
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'input_dim': self.input_dim,
                  'input_length': self.input_length}
        base_config = super(QRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [37]:
def qrnn_model(conv_layers = 2, max_dilation_rate = 3):
    embed_size = 128
    inp = Input(shape=(120, ))
    x = Embedding(50003, embed_size)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Conv1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = QRNN(128*2**(strides), return_sequences = True, stride = strides, dropout = 0.2, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = QRNN(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = QRNN(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10), go_backwards=True)(x)
    x = concatenate([x_f, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(2)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

    return model

qrnn_model = qrnn_model()
qrnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 120)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 120, 128)     6400384     input_6[0][0]                    
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 120, 128)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 118, 256)     98560       dropout_11[0][0]                 
__________________________________________________________________________________________________
conv1d_10 

In [38]:
batch_size = 512
epochs = 10

In [39]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks = [checkpoint, early_stopping]


In [ ]:
qrnn_model.fit(train_X,labels, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.20, callbacks=callbacks)


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 1526s 95ms/step - loss: 1.6681 - binary_accuracy: 0.4999 - val_loss: 0.7283 - val_binary_accuracy: 0.4999

Epoch 00001: val_loss improved from inf to 0.72834, saving model to early_weights.hdf5
Epoch 2/10
16000/16000 [==============================] - 1412s 88ms/step - loss: 0.7093 - binary_accuracy: 0.5397 - val_loss: 0.6385 - val_binary_accuracy: 0.5809

Epoch 00002: val_loss improved from 0.72834 to 0.63850, saving model to early_weights.hdf5
Epoch 3/10
16000/16000 [==============================] - 1401s 88ms/step - loss: 0.5560 - binary_accuracy: 0.7687 - val_loss: 1.2440 - val_binary_accuracy: 0.5179

Epoch 00003: val_loss did not improve
Epoch 4/10
16000/16000 [==============================] - 1402s 88ms/step - loss: nan - binary_accuracy: 0.0740 - val_loss: nan - val_binary_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve
Epoch 5/10
15872/16000 [===============

In [26]:
checkpoint = ModelCheckpoint('weight_sentiment_amazon.{epoch:02d}.hdf5')
callbacks_list = [checkpoint]

In [22]:
X_valid = train_X[:BATCH_SIZE]
Y_valid = labels[:BATCH_SIZE]
train_X = train_X[BATCH_SIZE:]
y_train_oh = labels[BATCH_SIZE:]

In [23]:
train_X = np.array(train_X)
X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)
y_train_oh = np.array(y_train_oh)


In [ ]:
train_X[0]
#X_valid.shape


In [27]:
history = model.fit(train_X, y_train_oh, validation_data=(X_valid, Y_valid), callbacks=callbacks_list, epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE)

Train on 19488 samples, validate on 512 samples
Epoch 1/26
19488/19488 [==============================] - 73s 4ms/step - loss: 0.6698 - acc: 0.5994 - val_loss: 0.5790 - val_acc: 0.7695
Epoch 2/26
19488/19488 [==============================] - 70s 4ms/step - loss: 0.4731 - acc: 0.7801 - val_loss: 0.3539 - val_acc: 0.8516
Epoch 3/26
19488/19488 [==============================] - 69s 4ms/step - loss: 0.3384 - acc: 0.8592 - val_loss: 0.3242 - val_acc: 0.8711
Epoch 4/26
19488/19488 [==============================] - 70s 4ms/step - loss: 0.2799 - acc: 0.8876 - val_loss: 0.3185 - val_acc: 0.8730
Epoch 5/26
19488/19488 [==============================] - 71s 4ms/step - loss: 0.2429 - acc: 0.9069 - val_loss: 0.3248 - val_acc: 0.8770
Epoch 6/26
19488/19488 [==============================] - 71s 4ms/step - loss: 0.2098 - acc: 0.9209 - val_loss: 0.3362 - val_acc: 0.8848
Epoch 7/26
19488/19488 [==============================] - 70s 4ms/step - loss: 0.1874 - acc: 0.9293 - val_loss: 0.3527 - val_acc: 

In [28]:
max(history.history['val_acc'])

0.884765625

In [31]:
def prep_data(inp):
    punct = ['!','"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', 
         '[', '/', ']', '^', '_', '`', '{', '|', '}', '~', '\t', '\n']
    #neg = negate_sequence(inp)
    for i in punct:
        inp.replace(i, '')
    tokenizer = TreebankWordTokenizer()
    inp = tokenizer.tokenize(inp)
    MAX_SEQ = 120
    n = MAX_SEQ - len(inp)
    if n < 0:
        inp = inp[:MAX_SEQ]
    else:
        for i in range(n):
            inp.append('<PAD>')
    for v in range(len(inp)):
        if inp[v] not in tk.word_index:
            inp[v] = tk.word_index['<UNK>']
        else:
            inp[v] = tk.word_index[inp[v]]
    return np.reshape(np.array(inp) , (1 , MAX_SEQ))

In [46]:
model.predict(prep_data("Hugely Disappointing: I've read all the previous entries in the Grantville Universe and enjoyed most of them quite a bit. But where the use of various writers and short stories worked well for a time, the quality of the writing has been steadily decreasing.The Ram Rebellion reads as a book totally abandoned by Eric Flint and handled almost entirely by the Regulars at Baen's Bar. I understand Baen's practice of pairing up-and-coming writers with more established writers in order to develop new talent and further the plot of popular series. It's worked well in the past.Here it simply failed. I'm about 1/5 through the book and am so tired of the lackluster writing, the sophomoric humor and the terribly slow plot progression that I'm putting it away for good. I'm probably unlikely to read the rest of the series because it's reached a point where having Flint's name on the cover is totally meaningless and not at all representative of the quality of writing to be expected within."))

array([[ 0.01202024,  0.98797977]], dtype=float32)

In [38]:
data_test = []
with open("train.ft.txt","r") as f:
    for i in f:
        data_test.append(i)

In [45]:
data_test[2000]

"__label__1 Hugely Disappointing: I've read all the previous entries in the Grantville Universe and enjoyed most of them quite a bit. But where the use of various writers and short stories worked well for a time, the quality of the writing has been steadily decreasing.The Ram Rebellion reads as a book totally abandoned by Eric Flint and handled almost entirely by the Regulars at Baen's Bar. I understand Baen's practice of pairing up-and-coming writers with more established writers in order to develop new talent and further the plot of popular series. It's worked well in the past.Here it simply failed. I'm about 1/5 through the book and am so tired of the lackluster writing, the sophomoric humor and the terribly slow plot progression that I'm putting it away for good. I'm probably unlikely to read the rest of the series because it's reached a point where having Flint's name on the cover is totally meaningless and not at all representative of the quality of writing to be expected within.

In [ ]:
dataa = []
with open("train.ft.txt","r") as f:
    for i in f:
        dataa.append(i)
        
len(dataa)